<a href="https://colab.research.google.com/github/kbrezinski/CS224W-GraphML/blob/main/notebooks/practice-GNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)

from platform import python_version
print(python_version())

1.11.0+cu113
3.7.13


In [2]:
!pip install torch-scatter torch-sparse \
 torch-cluster torch-spline-conv torch-geometric \
-f https://data.pyg.org/whl/torch-1.11.0+cu113.html -q
#!pip install ogb

     |████████████████████████████████| 7.9 MB 6.8 MB/s 
     |████████████████████████████████| 3.5 MB 70.9 MB/s 
     |████████████████████████████████| 2.5 MB 9.2 MB/s 
     |████████████████████████████████| 750 kB 51.2 MB/s 
     |████████████████████████████████| 407 kB 6.1 MB/s 
